# Extract Readings

In this tutorial we will show you how to use the CSVLoader class to load the readings table
from a folder that contains readings in the raw CSV format.

The Raw CSV format es briefly explained below, but more details can be found in [the documentation site](
https://signals-dev.github.io/GreenGuard/advanced_usage/csv.html)

During the next steps we will:

- Generate a folder with readings in the raw format based on the demo data
- Explore the raw format
- Load the redings needed for our target times
- Explore different options from the CSVLoader
- Load the readings in the unstacked format
- Store the readins and target times using pickle

## 0. Setup the logging

This step sets up logging in our environment to increase our visibility over
the steps that GreenGuard performs.

In [1]:
import logging;

logging.basicConfig(level=logging.INFO)
logging.getLogger().setLevel(level=logging.INFO)

import warnings
warnings.simplefilter("ignore")

## 1. Generate Raw Readings

The first step will be to execute the `generate_raw_readings` function, which will create a
folder in the indicated path and populate it with the raw version of the demo readings.

**NOTE**: if you want to use your own dataset you can skip this step and go directly to step 2.

In [2]:
from greenguard.demo import generate_raw_readings

target_times = generate_raw_readings('readings')

INFO:greenguard.demo:Generating file readings/T001/2013-01.csv
INFO:greenguard.demo:Generating file readings/T001/2013-02.csv
INFO:greenguard.demo:Generating file readings/T001/2013-03.csv
INFO:greenguard.demo:Generating file readings/T001/2013-04.csv
INFO:greenguard.demo:Generating file readings/T001/2013-05.csv
INFO:greenguard.demo:Generating file readings/T001/2013-06.csv
INFO:greenguard.demo:Generating file readings/T001/2013-07.csv
INFO:greenguard.demo:Generating file readings/T001/2013-08.csv
INFO:greenguard.demo:Generating file readings/T001/2013-09.csv
INFO:greenguard.demo:Generating file readings/T001/2013-10.csv
INFO:greenguard.demo:Generating file readings/T001/2013-11.csv
INFO:greenguard.demo:Generating file readings/T001/2013-12.csv


### Readings Format

Here we will load one of the generated CSV files to briefly explore its contents.

In [3]:
import pandas as pd

readings_sample = pd.read_csv('readings/T001/2013-01.csv')

In [4]:
readings_sample.head()

,signal_id,timestamp,value
0,S01,01/10/13 00:00:00,323.0
1,S02,01/10/13 00:00:00,320.0
2,S03,01/10/13 00:00:00,284.0
3,S04,01/10/13 00:00:00,348.0
4,S05,01/10/13 00:00:00,273.0


We can cleary see the format in which the data is stored:

* All the data from all the turbines is inside a single folder.
* Inside this folder, another folder exists for each turbine, named exactly like the turbine:
    * `readings/T001`
    * `readings/T002`
    * ...
* Inside each turbine folder one CSV file exists for each month, named `%Y-%m.csv`.
    * `readings/T001/2010-01.csv`
    * `readings/T001/2010-02.csv`
    * `readings/T001/2010-03.csv`
    * ...
* Each CSV file contains three columns:
    * `signal_id`: name or id of the signal.
    * ``timestamp``: timestamp of the reading formatted as ``%m/%d/%y %H:%M:%S``.
    * `value`: value of the reading.

### Target Times

The previous function will have also returned us a `target_times` variable,
which is a `pandas.DataFrame` containing the training examples, with the three expected columns:

* `turbine_id`: Id of the turbine associated with each training example
* `cutoff_time`: Time at which the prediction is being made
* `target`: Value that needs to be predicted

In [5]:
target_times.shape

(353, 3)

In [6]:
target_times.head()

,turbine_id,cutoff_time,target
0,T001,2013-01-12,0
1,T001,2013-01-13,0
2,T001,2013-01-14,0
3,T001,2013-01-15,1
4,T001,2013-01-16,0


In [7]:
target_times.target.mean()

0.3002832861189802

In [8]:
target_times.dtypes

turbine_id             object
cutoff_time    datetime64[ns]
target                  int64
dtype: object

## 2. CSVLoader

The readings in raw format can arbitrarily big, which might make it impossible to load
them into memory all at once.

In order to load them in an efficient way so that we can use them to solve Machine Learning
problems, GeenGuard provides the `greenguard.loaders.CVSLoader` class.

This class is prepared to, given a target times table, explore a collection of raw readings
and extract only the information needed to solve that particular problem.

The first step in order to use it is to create an instance passing it the path
to where the reading files are stored.

**NOTE**: If you want to use your own dataset instead of the demo version,
all you have to do is make the `readings_path` variable point at the
folder where you have your CVS files stored and load your `target_times` table:

Make sure to parse the `cutoff_time` column as a datetime!

```python
readings_path = 'path/to/your/data'
target_times = pd.read_csv('path/to/your/target_times.csv', parse_dates=['cutoff_time'])
```

In [9]:
from greenguard.loaders import CSVLoader

readings_path = 'readings'

csv_loader = CSVLoader(readings_path)

Once we have created our instance, we can load the readings needed for our target times by
calling the `load` method with the following two arguments:

* `target_times (pandas.DataFrame)`: the `target_times` table.
* `window_size (str)`: the size of the training window, as a timedelta specification
  (amount + time unit). This indicates the minimum amount of data that we need to
  load for each training example from the `target_times` table.
  
For example, let's load the readings needed for all our `target_times`, using a
`window_size` of **one day**.

In [10]:
new_target_times, readings = csv_loader.load(target_times, '1d')

INFO:greenguard.loaders.csv:Loaded 1306052 readings from turbine T001
INFO:greenguard.loaders.csv:Loaded 1306052 turbine readings
INFO:greenguard.targets:Dropped 0 targets without enough data. Final target_times size: 353


In [11]:
readings.shape

(1306052, 4)

In [12]:
readings.head()

,turbine_id,signal_id,timestamp,value
0,T001,S01,2013-01-11,209.0
1,T001,S02,2013-01-11,193.0
2,T001,S03,2013-01-11,177.0
3,T001,S04,2013-01-11,188.0
4,T001,S05,2013-01-11,150.0


In [13]:
readings.dtypes

turbine_id            object
signal_id             object
timestamp     datetime64[ns]
value                float64
dtype: object

We can see how the readings have been loaded with the expected format, including
the four expected columns:

* `turbine_id`: Unique identifier of the turbine which this reading comes from.
* `signal_id`: Unique identifier of the signal which this reading comes from.
* `timestamp (datetime)`: Time at which the reading took place, as a datetime.
* `value (float)`: Numerical value of this reading.

In [14]:
new_target_times.shape

(353, 3)

Let's see what happens if we increase the `window_size` to, for example, **30 days**.

In [15]:
new_target_times, readings = csv_loader.load(target_times, '30d')

INFO:greenguard.loaders.csv:Loaded 1309796 readings from turbine T001
INFO:greenguard.loaders.csv:Loaded 1309796 turbine readings
INFO:greenguard.targets:Dropped 28 targets without enough data. Final target_times size: 325


We can see now in the logged output above that there is a message that indicates that there
were 28 invalid targets that were dropped. This is because within our readings there was not
enough data to cover the entire training window for each traning example, so the ones that were
not covered were dropped to ensure that all the training examples are valid to work with them.

In [16]:
new_target_times.shape

(325, 3)

On the other side, we can see how now the size of the loaded readings table has increased,
as more data had to be included to properly cover all the training windows.

In [17]:
readings.shape

(1309796, 4)

## 3. Preprocessing the data

In some cases, if the amount of targets is big enough, loading high frequency data
into memory will still be a challenge.

For this cases, the `CSVLoader` class also supports passing a **resampling rule** and
an **aggregation function** specification. In this cases, the data will go through a
**sampling frequency reduction aggregation** while it is loaded, reducing the amount
of memory needed to load it.

In order to use the resampling feature, we will need to create a new instance of the
`CSVLoader` passing the following new arguments:

* `rule (str)`: Time-delta specification (amount+unit) of the new sampling frequency.
* `aggregation (str or function)`: Aggregation function to apply when resampling.

For example, let's create a `CSVLoader` instance that will reduce the sampling frequency
to **4 hours**, computing the **mean** of all the readings withing each interval.

In [18]:
csv_loader = CSVLoader(readings_path, rule='4h', aggregation='mean')

And then call the `load` method normally.

In [19]:
new_target_times, readings = csv_loader.load(target_times, '14d')

INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:81749 readings reduced to 3432
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:110938 readings reduced to 4680
INFO:greenguard.loaders.csv:112118 readings reduced to 4680
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:111862 readings reduced to 4680
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:114400 readings reduced to 4836
INFO:greenguard.loaders.csv:105321 readings reduced to 4550
INFO:greenguard.loaders.csv:108371 readings reduced to 4680
INFO:greenguard.loaders.csv:115615 readings reduced to 4836
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:115647 readings reduced to 4836
INFO:greeng

We can see now how the size of the readings table has been drastically reduced.

In [20]:
readings.shape

(55250, 4)

In [21]:
readings.head()

,turbine_id,signal_id,timestamp,value
0,T001,S01,2013-01-10 00:00:00,253.041667
1,T001,S01,2013-01-10 04:00:00,572.083333
2,T001,S01,2013-01-10 08:00:00,688.791667
3,T001,S01,2013-01-10 12:00:00,396.333333
4,T001,S01,2013-01-10 16:00:00,390.458333


In [22]:
new_target_times.shape

(341, 3)

## 4. Unstacking

Some of the pipelines included in **GreenGuard** expect a slightly different input format
where the data has been unstacked by `signal_id`, putting the values of each signal in a
different column instead of having all of them in a single one.

In such cases, the `CSVLoader` can also take care of the unstacking step.

For this, all you need to do is add `unstack=True` argument when creating the instance
and then use the `load` method as usual.

In [23]:
csv_loader = CSVLoader(readings_path, rule='4h', aggregation='mean', unstack=True)
new_target_times, readings = csv_loader.load(target_times, '14d')

INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:108371 readings reduced to 4680
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:115647 readings reduced to 4836
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:103319 readings reduced to 4368
INFO:greenguard.loaders.csv:115615 readings reduced to 4836
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:114400 readings reduced to 4836
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:114477 readings reduced to 4836
INFO:greenguard.loaders.csv:115979 readings reduced to 4836
INFO:greenguard.loaders.csv:111862 readings reduced to 4680
INFO:greenguard.loaders.csv:Resampling: 4h - mean
INFO:greenguard.loaders.csv:81749 readings reduced to 3432
INFO:greeng

The result is a table which has a much smaller number of rows, but one column for each signal

In [24]:
readings.shape

(2125, 28)

In [25]:
readings.head()

,turbine_id,timestamp,value_S01,value_S02,value_S03,value_S04,value_S05,value_S06,value_S07,value_S08,...,value_S17,value_S18,value_S19,value_S20,value_S21,value_S22,value_S23,value_S24,value_S25,value_S26
0,T001,2013-01-10 00:00:00,253.041667,268.250000,268.041667,297.166667,234.666667,261.916667,206.791667,3.198335e+06,...,9.079167,3.134510e+06,42.416667,44.958333,44.833333,49.625000,39.208333,43.833333,34.625,293.166667
1,T001,2013-01-10 04:00:00,572.083333,555.291667,538.666667,592.291667,557.166667,534.000000,544.250000,3.199514e+06,...,10.837500,3.142505e+06,62.083333,62.500000,63.625000,63.541667,61.333333,62.541667,54.000,421.208333
2,T001,2013-01-10 08:00:00,688.791667,696.791667,706.625000,750.791667,714.250000,683.333333,658.166667,3.201449e+06,...,12.754167,3.155809e+06,92.208333,94.958333,94.666667,97.333333,94.125000,93.583333,86.375,638.291667
3,T001,2013-01-10 12:00:00,396.333333,418.500000,415.791667,438.541667,382.250000,364.666667,320.333333,3.203319e+06,...,10.916667,3.168640e+06,55.750000,60.083333,58.583333,61.291667,52.791667,52.791667,44.000,376.125000
4,T001,2013-01-10 16:00:00,390.458333,408.875000,409.500000,458.000000,415.583333,363.000000,364.458333,3.204504e+06,...,10.412500,3.176672e+06,49.958333,53.875000,54.458333,56.750000,52.708333,46.708333,47.625,354.750000


## 5. Saving the readings

In some cases we will not be intending to use the generated `readings` and `target_times` tables
right away, but rather store them for later use.

### Using CSV

This can be done using pandas an plain `CSV` format:

**NOTE**: Notice the `index=False` argument. Otherwise, an extra index column will be added
to the CSV which would force us to modify the loading steps afterwards.

In [26]:
new_target_times.to_csv('my_problem_target_times.csv', index=False)

In [27]:
readings.to_csv('my_problem_readings.csv', index=False)

After this, we can easily reload the data back using pandas again.

**NOTE**: Notice how the datetime columns need to be passed so they can be parsed!

In [28]:
my_target_times = pd.read_csv('my_problem_target_times.csv', parse_dates=['cutoff_time'])

In [29]:
my_readings = pd.read_csv('my_problem_readings.csv', parse_dates=['timestamp'])

However, this has 2 inconvenients:
* Saving and loading the data is slow
* The datetimes need to be explicitly parsed

## Using Pickle

To solve the previously mentioned inconveniences we can use `pickle` instead of `CSV` format
to store our data.

In order to do this we will put the two tables in a `tuple` and store them using `pickle.dump`.

In [30]:
import pickle

with open('my_problem.plk', 'wb') as pickle_file:
    pickle.dump((new_target_times, readings), pickle_file)

And then load it back all at once using `pickle.load`.

In [31]:
with open('my_problem.plk', 'rb') as pickle_file:
    my_target_times, my_readings = pickle.load(pickle_file)